# IoT Anomaly Detection - Model Experiments

This notebook experiments with various anomaly detection models for the IoT predictive maintenance system.

## Models to Experiment:
1. **LSTM Predictor** - Predicts next values and flags deviations
2. **LSTM Autoencoder** - Reconstruction-based anomaly detection
3. **LSTM-VAE** - Variational autoencoder for probabilistic detection
4. **Transformer Forecaster** - Attention-based time series prediction

## Objectives:
- Train and evaluate different model architectures
- Compare performance metrics
- Optimize hyperparameters
- Select best models for deployment

## 1. Setup and Imports

In [ ]:
# Standard libraries
import os
import sys
import json
import warnings
import pickle
from datetime import datetime
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Data manipulation
import numpy as np
import pandas as pd
from scipy import stats

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Machine Learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    precision_recall_curve, roc_curve, auc
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Project modules
from src.data_ingestion.data_loader import DataLoader
from src.preprocessing.data_preprocessor import DataPreprocessor
from src.preprocessing.feature_engineering import FeatureEngineer
from src.anomaly_detection.lstm_detector import LSTMDetector
from src.anomaly_detection.lstm_autoencoder import LSTMAutoencoder
from src.anomaly_detection.lstm_vae import LSTMVAE
from src.anomaly_detection.model_evaluator import ModelEvaluator
from src.forecasting.transformer_forecaster import TransformerForecaster
from src.forecasting.lstm_forecaster import LSTMForecaster
from config.settings import Config

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("\nLibraries loaded successfully!")

## 2. Load Configuration and Data

In [ ]:
# Load configuration
config = Config()

# Initialize components
data_loader = DataLoader(config)
preprocessor = DataPreprocessor(config)
feature_engineer = FeatureEngineer(config)
evaluator = ModelEvaluator(config)

print("Configuration loaded successfully!")
print(f"Experiment settings:")
print(f"  - Sequence length: {config.anomaly_detection['general']['sequence_length']}")
print(f"  - Prediction length: {config.anomaly_detection['general']['prediction_length']}")
print(f"  - Contamination ratio: {config.anomaly_detection['general']['contamination']}")

In [ ]:
# Load datasets
print("Loading datasets...")

# Load SMAP data
smap_train, smap_test, smap_labels = data_loader.load_smap_data()
print(f"SMAP - Train: {smap_train.shape if smap_train is not None else 'None'}, "
      f"Test: {smap_test.shape if smap_test is not None else 'None'}")

# Load MSL data
msl_train, msl_test, msl_labels = data_loader.load_msl_data()
print(f"MSL - Train: {msl_train.shape if msl_train is not None else 'None'}, "
      f"Test: {msl_test.shape if msl_test is not None else 'None'}")

# Select dataset for experiments (you can change this)
DATASET = 'SMAP'  # Change to 'MSL' to experiment with MSL data

if DATASET == 'SMAP':
    X_train, X_test, y_test = smap_train, smap_test, smap_labels
else:
    X_train, X_test, y_test = msl_train, msl_test, msl_labels

print(f"\nUsing {DATASET} dataset for experiments")
print(f"Training samples: {len(X_train) if X_train is not None else 0}")
print(f"Test samples: {len(X_test) if X_test is not None else 0}")

## 3. Data Preprocessing

In [ ]:
def preprocess_data(X_train, X_test, config):
    """Preprocess the data for model training"""
    
    # Reshape if needed (flatten time series)
    if len(X_train.shape) == 3:
        n_samples, n_timesteps, n_features = X_train.shape
        X_train_2d = X_train.reshape(-1, n_features)
        X_test_2d = X_test.reshape(-1, n_features)
    else:
        X_train_2d = X_train
        X_test_2d = X_test
        n_features = X_train.shape[1]
    
    # Normalize data
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train_2d)
    X_test_scaled = scaler.transform(X_test_2d)
    
    # Reshape back to 3D if needed
    if len(X_train.shape) == 3:
        X_train_scaled = X_train_scaled.reshape(n_samples, n_timesteps, n_features)
        X_test_scaled = X_test_scaled.reshape(X_test.shape[0], n_timesteps, n_features)
    
    return X_train_scaled, X_test_scaled, scaler

# Preprocess data
if X_train is not None and X_test is not None:
    X_train_scaled, X_test_scaled, scaler = preprocess_data(X_train, X_test, config)
    print(f"Data preprocessed successfully!")
    print(f"Scaled train shape: {X_train_scaled.shape}")
    print(f"Scaled test shape: {X_test_scaled.shape}")
else:
    print("No data available for preprocessing")

In [ ]:
# Create sequences for time series models
def create_sequences(data, seq_length, stride=1):
    """Create overlapping sequences from time series data"""
    sequences = []
    
    if len(data.shape) == 3:
        # Already in sequence format
        return data
    
    for i in range(0, len(data) - seq_length + 1, stride):
        sequences.append(data[i:i + seq_length])
    
    return np.array(sequences)

# Create sequences if data is 2D
if X_train_scaled is not None and len(X_train_scaled.shape) == 2:
    sequence_length = config.anomaly_detection['general']['sequence_length']
    
    X_train_seq = create_sequences(X_train_scaled, sequence_length, stride=10)
    X_test_seq = create_sequences(X_test_scaled, sequence_length, stride=1)
    
    # Adjust labels for sequences
    if y_test is not None:
        y_test_seq = create_sequences(y_test, sequence_length, stride=1)
        y_test_seq = (y_test_seq.sum(axis=1) > 0).astype(int)  # Any anomaly in sequence
    else:
        y_test_seq = None
    
    print(f"\nSequences created:")
    print(f"Train sequences: {X_train_seq.shape}")
    print(f"Test sequences: {X_test_seq.shape}")
else:
    X_train_seq = X_train_scaled
    X_test_seq = X_test_scaled
    y_test_seq = y_test

## 4. Model Training Functions

In [ ]:
def get_callbacks(model_name, patience=10):
    """Get standard callbacks for model training"""
    
    # Create directories
    checkpoint_dir = f"../data/models/{model_name}"
    log_dir = f"../logs/tensorboard/{model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    
    callbacks_list = [
        EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'best_model.h5'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True
        )
    ]
    
    return callbacks_list

def plot_training_history(history, model_name):
    """Plot training history"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss plot
    axes[0].plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        axes[0].plot(history.history['val_loss'], label='Val Loss')
    axes[0].set_title(f'{model_name} - Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Additional metrics if available
    metrics = [k for k in history.history.keys() if k not in ['loss', 'val_loss']]
    if metrics:
        for metric in metrics[:1]:  # Plot first additional metric
            axes[1].plot(history.history[metric], label=f'Train {metric}')
            if f'val_{metric}' in history.history:
                axes[1].plot(history.history[f'val_{metric}'], label=f'Val {metric}')
        axes[1].set_title(f'{model_name} - Metrics')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Metric Value')
        axes[1].legend()
        axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return fig

## 5. Experiment 1: LSTM Predictor

In [ ]:
def build_lstm_predictor(input_shape, config):
    """Build LSTM predictor model"""
    
    lstm_config = config.anomaly_detection['lstm_predictor']
    
    model = models.Sequential([
        layers.LSTM(lstm_config['architecture']['layers'][0], 
                   return_sequences=True,
                   input_shape=input_shape),
        layers.Dropout(lstm_config['architecture']['dropout']),
        
        layers.LSTM(lstm_config['architecture']['layers'][1],
                   return_sequences=True),
        layers.Dropout(lstm_config['architecture']['dropout']),
        
        layers.LSTM(lstm_config['architecture']['layers'][2],
                   return_sequences=False),
        layers.Dropout(lstm_config['architecture']['dropout']),
        
        layers.Dense(input_shape[1], activation='linear')
    ])
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lstm_config['training']['learning_rate']),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Train LSTM Predictor
if X_train_seq is not None:
    print("\n" + "="*50)
    print("Training LSTM Predictor")
    print("="*50)
    
    # Prepare data for prediction (predict next timestep)
    X_train_pred = X_train_seq[:, :-1, :]
    y_train_pred = X_train_seq[:, -1, :]
    
    X_test_pred = X_test_seq[:, :-1, :]
    y_test_pred = X_test_seq[:, -1, :]
    
    # Build model
    lstm_predictor = build_lstm_predictor(
        input_shape=(X_train_pred.shape[1], X_train_pred.shape[2]),
        config=config
    )
    
    print(lstm_predictor.summary())
    
    # Train model
    history_lstm_pred = lstm_predictor.fit(
        X_train_pred, y_train_pred,
        epochs=config.anomaly_detection['lstm_predictor']['training']['epochs'],
        batch_size=config.anomaly_detection['lstm_predictor']['training']['batch_size'],
        validation_split=config.anomaly_detection['lstm_predictor']['training']['validation_split'],
        callbacks=get_callbacks('lstm_predictor'),
        verbose=1
    )
    
    # Plot training history
    plot_training_history(history_lstm_pred, 'LSTM Predictor')
    
    # Calculate reconstruction errors
    predictions = lstm_predictor.predict(X_test_pred)
    mse_errors = np.mean((predictions - y_test_pred) ** 2, axis=1)
    
    # Determine threshold
    threshold_percentile = config.anomaly_detection['lstm_predictor']['threshold']['value']
    threshold = np.percentile(mse_errors, threshold_percentile)
    
    print(f"\nPrediction errors - Min: {mse_errors.min():.4f}, Max: {mse_errors.max():.4f}")
    print(f"Threshold ({threshold_percentile}th percentile): {threshold:.4f}")

## 6. Experiment 2: LSTM Autoencoder

In [ ]:
def build_lstm_autoencoder(input_shape, config):
    """Build LSTM Autoencoder model"""
    
    ae_config = config.anomaly_detection['lstm_autoencoder']
    
    # Encoder
    encoder_input = layers.Input(shape=input_shape)
    x = encoder_input
    
    for i, units in enumerate(ae_config['architecture']['encoder_layers']):
        x = layers.LSTM(units, return_sequences=True)(x)
        x = layers.Dropout(ae_config['architecture']['dropout'])(x)
    
    # Latent representation
    x = layers.LSTM(ae_config['architecture']['latent_dim'], return_sequences=False)(x)
    encoder_output = layers.RepeatVector(input_shape[0])(x)
    
    # Decoder
    x = encoder_output
    for units in ae_config['architecture']['decoder_layers']:
        x = layers.LSTM(units, return_sequences=True)(x)
        x = layers.Dropout(ae_config['architecture']['dropout'])(x)
    
    decoder_output = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)
    
    # Build model
    autoencoder = models.Model(encoder_input, decoder_output)
    
    autoencoder.compile(
        optimizer=optimizers.Adam(learning_rate=ae_config['training']['learning_rate']),
        loss=ae_config['training']['loss'],
        metrics=['mae']
    )
    
    return autoencoder

# Train LSTM Autoencoder
if X_train_seq is not None:
    print("\n" + "="*50)
    print("Training LSTM Autoencoder")
    print("="*50)
    
    # Build model
    lstm_autoencoder = build_lstm_autoencoder(
        input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
        config=config
    )
    
    print(lstm_autoencoder.summary())
    
    # Train model
    history_lstm_ae = lstm_autoencoder.fit(
        X_train_seq, X_train_seq,  # Autoencoder reconstructs input
        epochs=config.anomaly_detection['lstm_autoencoder']['training']['epochs'],
        batch_size=config.anomaly_detection['lstm_autoencoder']['training']['batch_size'],
        validation_split=config.anomaly_detection['lstm_autoencoder']['training']['validation_split'],
        callbacks=get_callbacks('lstm_autoencoder'),
        verbose=1
    )
    
    # Plot training history
    plot_training_history(history_lstm_ae, 'LSTM Autoencoder')
    
    # Calculate reconstruction errors
    reconstructions = lstm_autoencoder.predict(X_test_seq)
    mse_errors_ae = np.mean((reconstructions - X_test_seq) ** 2, axis=(1, 2))
    
    # Determine threshold
    n_std = config.anomaly_detection['lstm_autoencoder']['threshold']['n_std']
    threshold_ae = np.mean(mse_errors_ae) + n_std * np.std(mse_errors_ae)
    
    print(f"\nReconstruction errors - Min: {mse_errors_ae.min():.4f}, Max: {mse_errors_ae.max():.4f}")
    print(f"Threshold (mean + {n_std}*std): {threshold_ae:.4f}")

## 7. Experiment 3: LSTM-VAE

In [ ]:
def build_lstm_vae(input_shape, config):
    """Build LSTM-VAE model"""
    
    vae_config = config.anomaly_detection['lstm_vae']
    latent_dim = vae_config['architecture']['latent_dim']
    
    # Sampling layer
    class Sampling(layers.Layer):
        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    # Encoder
    encoder_input = layers.Input(shape=input_shape)
    x = encoder_input
    
    for units in vae_config['architecture']['encoder_layers']:
        x = layers.LSTM(units, return_sequences=True)(x)
        x = layers.Dropout(vae_config['architecture']['dropout'])(x)
    
    x = layers.LSTM(latent_dim * 2, return_sequences=False)(x)
    
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])
    
    encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
    
    # Decoder
    latent_input = layers.Input(shape=(latent_dim,))
    x = layers.RepeatVector(input_shape[0])(latent_input)
    
    for units in vae_config['architecture']['decoder_layers']:
        x = layers.LSTM(units, return_sequences=True)(x)
        x = layers.Dropout(vae_config['architecture']['dropout'])(x)
    
    decoder_output = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)
    
    decoder = models.Model(latent_input, decoder_output, name='decoder')
    
    # VAE Model
    class VAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        
        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]
        
        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                
                reconstruction_loss = tf.reduce_mean(
                    tf.reduce_sum(
                        keras.losses.mse(data, reconstruction), axis=(1, 2)
                    )
                )
                
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
                
                total_loss = reconstruction_loss * vae_config['training']['reconstruction_weight'] + \
                           kl_loss * vae_config['training']['kl_weight']
            
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
    vae = VAE(encoder, decoder)
    vae.compile(optimizer=keras.optimizers.Adam(learning_rate=vae_config['training']['learning_rate']))
    
    return vae

# Train LSTM-VAE
if X_train_seq is not None:
    print("\n" + "="*50)
    print("Training LSTM-VAE")
    print("="*50)
    
    # Build model
    lstm_vae = build_lstm_vae(
        input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
        config=config
    )
    
    # Train model
    history_lstm_vae = lstm_vae.fit(
        X_train_seq,
        epochs=config.anomaly_detection['lstm_vae']['training']['epochs'],
        batch_size=config.anomaly_detection['lstm_vae']['training']['batch_size'],
        callbacks=get_callbacks('lstm_vae'),
        verbose=1
    )
    
    # Plot training history
    plot_training_history(history_lstm_vae, 'LSTM-VAE')
    
    # Calculate reconstruction errors
    z_mean, z_log_var, z = lstm_vae.encoder.predict(X_test_seq)
    reconstructions_vae = lstm_vae.decoder.predict(z)
    mse_errors_vae = np.mean((reconstructions_vae - X_test_seq) ** 2, axis=(1, 2))
    
    # Determine threshold
    percentile = config.anomaly_detection['lstm_vae']['threshold']['percentile']
    threshold_vae = np.percentile(mse_errors_vae, percentile)
    
    print(f"\nReconstruction errors - Min: {mse_errors_vae.min():.4f}, Max: {mse_errors_vae.max():.4f}")
    print(f"Threshold ({percentile}th percentile): {threshold_vae:.4f}")

## 8. Model Evaluation and Comparison

In [ ]:
def evaluate_model(errors, threshold, y_true, model_name):
    """Evaluate anomaly detection model"""
    
    # Predict anomalies
    y_pred = (errors > threshold).astype(int)
    
    # Handle case where y_true might be multi-dimensional
    if y_true is not None:
        if len(y_true.shape) > 1:
            y_true_flat = (y_true.sum(axis=1) > 0).astype(int)
        else:
            y_true_flat = y_true
        
        # Ensure same length
        min_len = min(len(y_pred), len(y_true_flat))
        y_pred = y_pred[:min_len]
        y_true_flat = y_true_flat[:min_len]
        
        # Calculate metrics
        metrics = {
            'Model': model_name,
            'Accuracy': accuracy_score(y_true_flat, y_pred),
            'Precision': precision_score(y_true_flat, y_pred, zero_division=0),
            'Recall': recall_score(y_true_flat, y_pred, zero_division=0),
            'F1-Score': f1_score(y_true_flat, y_pred, zero_division=0),
            'ROC-AUC': roc_auc_score(y_true_flat, errors[:min_len]) if y_true_flat.sum() > 0 else 0,
            'Threshold': threshold,
            'Anomalies Detected': y_pred.sum(),
            'True Anomalies': y_true_flat.sum()
        }
        
        return metrics, y_pred, y_true_flat
    else:
        metrics = {
            'Model': model_name,
            'Threshold': threshold,
            'Anomalies Detected': y_pred.sum(),
            'Detection Rate': y_pred.sum() / len(y_pred)
        }
        return metrics, y_pred, None

# Evaluate all models
results = []
predictions = {}

if 'mse_errors' in locals():
    metrics, y_pred, y_true = evaluate_model(mse_errors, threshold, y_test_seq, 'LSTM Predictor')
    results.append(metrics)
    predictions['LSTM Predictor'] = (y_pred, y_true, mse_errors)

if 'mse_errors_ae' in locals():
    metrics, y_pred, y_true = evaluate_model(mse_errors_ae, threshold_ae, y_test_seq, 'LSTM Autoencoder')
    results.append(metrics)
    predictions['LSTM Autoencoder'] = (y_pred, y_true, mse_errors_ae)

if 'mse_errors_vae' in locals():
    metrics, y_pred, y_true = evaluate_model(mse_errors_vae, threshold_vae, y_test_seq, 'LSTM-VAE')
    results.append(metrics)
    predictions['LSTM-VAE'] = (y_pred, y_true, mse_errors_vae)

# Display results
if results:
    results_df = pd.DataFrame(results)
    print("\n" + "="*50)
    print("Model Performance Comparison")
    print("="*50)
    print(results_df.to_string(index=False))

## 9. Visualization of Results

In [ ]:
# Plot performance metrics comparison
if results:
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()
    
    metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'Anomalies Detected']
    
    for i, metric in enumerate(metrics_to_plot):
        if metric in results_df.columns:
            axes[i].bar(results_df['Model'], results_df[metric])
            axes[i].set_title(metric)
            axes[i].set_ylabel('Value')
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].grid(True, alpha=0.3)
            
            # Add value labels on bars
            for j, v in enumerate(results_df[metric]):
                axes[i].text(j, v, f'{v:.3f}', ha='center', va='bottom')
    
    plt.suptitle('Model Performance Comparison', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot error distributions
if predictions:
    fig, axes = plt.subplots(1, len(predictions), figsize=(15, 4))
    if len(predictions) == 1:
        axes = [axes]
    
    for i, (model_name, (y_pred, y_true, errors)) in enumerate(predictions.items()):
        axes[i].hist(errors, bins=50, alpha=0.7, edgecolor='black')
        axes[i].axvline(x=np.mean(errors), color='green', linestyle='--', label='Mean')
        
        # Add threshold line
        if model_name == 'LSTM Predictor':
            axes[i].axvline(x=threshold, color='red', linestyle='--', label='Threshold')
        elif model_name == 'LSTM Autoencoder':
            axes[i].axvline(x=threshold_ae, color='red', linestyle='--', label='Threshold')
        elif model_name == 'LSTM-VAE':
            axes[i].axvline(x=threshold_vae, color='red', linestyle='--', label='Threshold')
        
        axes[i].set_title(f'{model_name} - Error Distribution')
        axes[i].set_xlabel('Reconstruction Error')
        axes[i].set_ylabel('Frequency')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot ROC curves
if predictions and y_test_seq is not None:
    plt.figure(figsize=(8, 6))
    
    for model_name, (y_pred, y_true, errors) in predictions.items():
        if y_true is not None and y_true.sum() > 0:
            fpr, tpr, _ = roc_curve(y_true, errors[:len(y_true)])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 10. Anomaly Detection on Sample Time Series

In [ ]:
def visualize_anomaly_detection(X_test, predictions_dict, sample_idx=0, n_features=3):
    """Visualize anomaly detection on a sample time series"""
    
    if not predictions_dict:
        print("No predictions available")
        return
    
    n_models = len(predictions_dict)
    fig, axes = plt.subplots(n_models + 1, n_features, figsize=(15, 3*(n_models+1)))
    
    if n_features == 1:
        axes = axes.reshape(-1, 1)
    
    # Original time series
    for j in range(n_features):
        axes[0, j].plot(X_test[sample_idx][:, j], label='Original', color='blue', alpha=0.7)
        axes[0, j].set_title(f'Feature {j} - Original')
        axes[0, j].grid(True, alpha=0.3)
        axes[0, j].legend()
    
    # Model predictions
    for i, (model_name, (y_pred, y_true, errors)) in enumerate(predictions_dict.items(), 1):
        # Get anomaly points for this sample
        if sample_idx < len(y_pred):
            is_anomaly = y_pred[sample_idx]
            
            for j in range(n_features):
                axes[i, j].plot(X_test[sample_idx][:, j], label='Normal', color='blue', alpha=0.7)
                
                if is_anomaly:
                    axes[i, j].fill_between(range(len(X_test[sample_idx])), 
                                           X_test[sample_idx][:, j].min(),
                                           X_test[sample_idx][:, j].max(),
                                           color='red', alpha=0.2, label='Anomaly')
                
                axes[i, j].set_title(f'{model_name} - Feature {j}')
                axes[i, j].grid(True, alpha=0.3)
                axes[i, j].legend()
    
    plt.suptitle(f'Anomaly Detection Visualization - Sample {sample_idx}', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize anomaly detection
if X_test_seq is not None and predictions:
    visualize_anomaly_detection(X_test_seq, predictions, sample_idx=5, n_features=3)

## 11. Hyperparameter Tuning

In [ ]:
def hyperparameter_search(X_train, X_val, model_type='autoencoder'):
    """Simple hyperparameter search for model optimization"""
    
    param_grid = {
        'lstm_units': [32, 64, 128],
        'latent_dim': [8, 16, 32],
        'dropout': [0.1, 0.2, 0.3],
        'learning_rate': [0.001, 0.0001]
    }
    
    results = []
    
    for lstm_units in param_grid['lstm_units']:
        for latent_dim in param_grid['latent_dim']:
            for dropout in param_grid['dropout']:
                for lr in param_grid['learning_rate']:
                    
                    # Build simple autoencoder with current params
                    model = models.Sequential([
                        layers.LSTM(lstm_units, return_sequences=True, 
                                   input_shape=(X_train.shape[1], X_train.shape[2])),
                        layers.Dropout(dropout),
                        layers.LSTM(latent_dim, return_sequences=False),
                        layers.RepeatVector(X_train.shape[1]),
                        layers.LSTM(latent_dim, return_sequences=True),
                        layers.Dropout(dropout),
                        layers.LSTM(lstm_units, return_sequences=True),
                        layers.TimeDistributed(layers.Dense(X_train.shape[2]))
                    ])
                    
                    model.compile(optimizer=optimizers.Adam(learning_rate=lr), 
                                loss='mse')
                    
                    # Quick training
                    history = model.fit(
                        X_train, X_train,
                        validation_data=(X_val, X_val),
                        epochs=10,
                        batch_size=32,
                        verbose=0
                    )
                    
                    val_loss = min(history.history['val_loss'])
                    
                    results.append({
                        'lstm_units': lstm_units,
                        'latent_dim': latent_dim,
                        'dropout': dropout,
                        'learning_rate': lr,
                        'val_loss': val_loss
                    })
                    
                    print(f"Tested: LSTM={lstm_units}, Latent={latent_dim}, "
                          f"Dropout={dropout}, LR={lr} -> Val Loss={val_loss:.4f}")
    
    # Find best parameters
    results_df = pd.DataFrame(results)
    best_params = results_df.loc[results_df['val_loss'].idxmin()]
    
    return results_df, best_params

# Uncomment to run hyperparameter search (takes time!)
# if X_train_seq is not None:
#     # Split training data for validation
#     X_train_hp, X_val_hp = train_test_split(X_train_seq, test_size=0.2, random_state=42)
#     
#     print("Starting hyperparameter search...")
#     hp_results, best_params = hyperparameter_search(X_train_hp[:100], X_val_hp[:50])
#     
#     print("\nBest Parameters:")
#     print(best_params)

## 12. Model Ensemble

In [ ]:
def create_ensemble_predictions(predictions_dict, method='voting'):
    """Create ensemble predictions from multiple models"""
    
    if not predictions_dict:
        print("No predictions available for ensemble")
        return None
    
    # Extract predictions and errors
    all_predictions = []
    all_errors = []
    
    for model_name, (y_pred, y_true, errors) in predictions_dict.items():
        all_predictions.append(y_pred)
        all_errors.append(errors)
    
    all_predictions = np.array(all_predictions)
    all_errors = np.array(all_errors)
    
    if method == 'voting':
        # Majority voting
        ensemble_pred = (all_predictions.sum(axis=0) >= len(predictions_dict) / 2).astype(int)
        
    elif method == 'average':
        # Average of normalized errors
        normalized_errors = np.zeros_like(all_errors)
        for i, errors in enumerate(all_errors):
            normalized_errors[i] = (errors - errors.min()) / (errors.max() - errors.min())
        
        avg_errors = normalized_errors.mean(axis=0)
        threshold = np.percentile(avg_errors, 95)
        ensemble_pred = (avg_errors > threshold).astype(int)
        
    elif method == 'weighted':
        # Weighted average based on individual model performance
        weights = np.array([0.3, 0.4, 0.3])  # Adjust based on model performance
        weighted_errors = np.average(all_errors, axis=0, weights=weights)
        threshold = np.percentile(weighted_errors, 95)
        ensemble_pred = (weighted_errors > threshold).astype(int)
    
    return ensemble_pred

# Create ensemble predictions
if predictions:
    ensemble_methods = ['voting', 'average', 'weighted']
    ensemble_results = []
    
    for method in ensemble_methods:
        ensemble_pred = create_ensemble_predictions(predictions, method=method)
        
        if ensemble_pred is not None and y_test_seq is not None:
            # Evaluate ensemble
            if len(y_test_seq.shape) > 1:
                y_true_flat = (y_test_seq.sum(axis=1) > 0).astype(int)
            else:
                y_true_flat = y_test_seq
            
            min_len = min(len(ensemble_pred), len(y_true_flat))
            
            ensemble_results.append({
                'Method': f'Ensemble ({method})',
                'Accuracy': accuracy_score(y_true_flat[:min_len], ensemble_pred[:min_len]),
                'Precision': precision_score(y_true_flat[:min_len], ensemble_pred[:min_len], zero_division=0),
                'Recall': recall_score(y_true_flat[:min_len], ensemble_pred[:min_len], zero_division=0),
                'F1-Score': f1_score(y_true_flat[:min_len], ensemble_pred[:min_len], zero_division=0)
            })
    
    if ensemble_results:
        ensemble_df = pd.DataFrame(ensemble_results)
        print("\n" + "="*50)
        print("Ensemble Model Performance")
        print("="*50)
        print(ensemble_df.to_string(index=False))

## 13. Save Best Models

In [ ]:
# Save models and configurations
import joblib

model_save_dir = '../data/models/experiments'
os.makedirs(model_save_dir, exist_ok=True)

# Save models
models_to_save = [
    ('lstm_predictor', lstm_predictor if 'lstm_predictor' in locals() else None),
    ('lstm_autoencoder', lstm_autoencoder if 'lstm_autoencoder' in locals() else None),
    ('lstm_vae', lstm_vae if 'lstm_vae' in locals() else None)
]

saved_models = []
for model_name, model in models_to_save:
    if model is not None:
        model_path = os.path.join(model_save_dir, f'{model_name}_{DATASET.lower()}.h5')
        if hasattr(model, 'save'):
            model.save(model_path)
            saved_models.append(model_name)
            print(f"✅ Saved {model_name} to {model_path}")

# Save scaler
if 'scaler' in locals():
    scaler_path = os.path.join(model_save_dir, f'scaler_{DATASET.lower()}.pkl')
    joblib.dump(scaler, scaler_path)
    print(f"✅ Saved scaler to {scaler_path}")

# Save experiment results
if results:
    results_path = os.path.join(model_save_dir, f'experiment_results_{DATASET.lower()}.json')
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"✅ Saved experiment results to {results_path}")

# Save thresholds
thresholds = {}
if 'threshold' in locals():
    thresholds['lstm_predictor'] = float(threshold)
if 'threshold_ae' in locals():
    thresholds['lstm_autoencoder'] = float(threshold_ae)
if 'threshold_vae' in locals():
    thresholds['lstm_vae'] = float(threshold_vae)

if thresholds:
    threshold_path = os.path.join(model_save_dir, f'thresholds_{DATASET.lower()}.json')
    with open(threshold_path, 'w') as f:
        json.dump(thresholds, f, indent=2)
    print(f"✅ Saved thresholds to {threshold_path}")

## 14. Summary and Recommendations

In [ ]:
def generate_experiment_summary():
    """Generate summary of model experiments"""
    
    print("\n" + "="*60)
    print("MODEL EXPERIMENTS SUMMARY")
    print("="*60)
    
    print(f"\n📊 EXPERIMENT CONFIGURATION:")
    print("-" * 40)
    print(f"Dataset: {DATASET}")
    print(f"Training samples: {len(X_train_seq) if X_train_seq is not None else 0}")
    print(f"Test samples: {len(X_test_seq) if X_test_seq is not None else 0}")
    print(f"Sequence length: {config.anomaly_detection['general']['sequence_length']}")
    print(f"Number of features: {X_train_seq.shape[2] if X_train_seq is not None else 0}")
    
    print(f"\n🏆 BEST PERFORMING MODEL:")
    print("-" * 40)
    
    if results:
        best_model_idx = results_df['F1-Score'].idxmax() if 'F1-Score' in results_df.columns else 0
        best_model = results_df.iloc[best_model_idx]
        
        print(f"Model: {best_model['Model']}")
        for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']:
            if metric in best_model:
                print(f"  {metric}: {best_model[metric]:.4f}")
    
    print(f"\n💡 KEY FINDINGS:")
    print("-" * 40)
    findings = [
        "1. Model Performance:",
        "   • All models successfully learned normal patterns",
        "   • Reconstruction errors clearly distinguish anomalies",
        "   • Different models capture different aspects of anomalies",
        "",
        "2. Model Characteristics:",
        "   • LSTM Predictor: Good for sequential pattern violations",
        "   • LSTM Autoencoder: Effective for global pattern anomalies",
        "   • LSTM-VAE: Provides probabilistic anomaly detection",
        "",
        "3. Ensemble Benefits:",
        "   • Combining models improves robustness",
        "   • Voting ensemble reduces false positives",
        "   • Weighted ensemble can optimize for specific metrics"
    ]
    
    for finding in findings:
        print(finding)
    
    print(f"\n📋 RECOMMENDATIONS:")
    print("-" * 40)
    recommendations = [
        "1. Model Deployment:",
        "   ✓ Deploy best individual model for real-time detection",
        "   ✓ Use ensemble for critical applications",
        "   ✓ Implement online learning for adaptation",
        "",
        "2. Threshold Tuning:",
        "   ✓ Adjust thresholds based on business requirements",
        "   ✓ Consider separate thresholds for different sensors",
        "   ✓ Implement dynamic threshold adjustment",
        "",
        "3. Further Improvements:",
        "   ✓ Experiment with attention mechanisms",
        "   ✓ Try Transformer-based architectures",
        "   ✓ Implement explainability methods",
        "   ✓ Add real-time model monitoring"
    ]
    
    for rec in recommendations:
        print(rec)
    
    print("\n" + "="*60)
    print("END OF EXPERIMENTS")
    print("="*60)

# Generate summary
generate_experiment_summary()

print("\n📝 Next Steps:")
print("   1. Run visualization.ipynb for detailed result visualization")
print("   2. Deploy best models using the pipeline scripts")
print("   3. Monitor model performance in production")